# **Lab 03: Khai thác và trực quan hóa dữ liệu**

### **Danh sách các thành viên**

<center>

|         Họ tên         |   MSSV   |
| :--------------------- | :------: |
| Trần Nguyễn Nhật Cường | 22127048 |
|    Nguyễn Công Tuấn    | 22127436 |
|     Trần Đăng Tuấn     | 22127438 |

</center>

## **Cài đặt và sử dụng các thư viện cần thiết**

In [ ]:
! pip install matplotlib pandas seaborn wbgapi scikit-learn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import wbgapi as wb

pd.options.display.float_format = '{:.2f}'.format

**`matplotlib`** được dùng để vẽ biểu đồ trong Python. Có thể dùng để vẽ các loại biểu đồ như
histogram, biểu đồ tán xạ (scatter plot), biểu đồ đường (line chart) và biểu đồ cột (bar chart),...

**`pandas`** được dùng để xử lý và phân tích dữ liệu dạng bảng và cung cấp cấu trúc dữ liệu như
DataFrame và Series để dễ thao tác

**`seaborn`** dùng để vẽ biểu đồ dựa trên thư viện matplotlib nhưng biểu đồ sẽ trông đẹp hơn và cũng hỗ trợ các loại biểu đồ thống kê như violin plot, box plot, heatmap

**`wbgapi`** dùng để lấy/truy cập dữ liệu từ ngân hàng thế giới (World Bank).

**`sklearn (scikit-learn)`** là thư viện học máy phổ biến trong Python, được sử dụng để thực hiện các tác vụ học máy và xử lý dữ liệu. Trong đồ án này, các thành phần sau của sklearn được sử dụng:
- **`sklearn.cluster.KMeans`**: Cung cấp thuật toán KMeans để gom nhóm dữ liệu thành các cụm dựa trên sự tương đồng, được sử dụng để phân loại các nền kinh tế thành các nhóm.
- **`sklearn.metrics.silhouette_score`**: Dùng để tính điểm Silhouette, một chỉ số đánh giá chất lượng của việc gom nhóm, giúp xác định số lượng cụm tối ưu bằng cách đo lường mức độ tách biệt giữa các cụm.
- **`sklearn.preprocessing.StandardScaler`**: Dùng để chuẩn hóa dữ liệu, đưa các giá trị về cùng thang đo (trung bình bằng 0 và độ lệch chuẩn bằng 1), đảm bảo các chỉ số kinh tế có trọng số công bằng khi áp dụng thuật toán gom nhóm.

## **Thu thập dữ liệu**

**Nền tảng thu thập** - *World Development Indicators* từ *World Bank*

**Phương pháp thư thập** - Gọi *APIs* từ **WBGAPI** - một package để truy cập vào dữ liệu *World Bank* - *Ngân hàng thế giới*

**Cách lưu trữ dữ liệu sau khi thu thập** - Lưu thành file **`wdi_2000_2020_VNM`**, **`wdi_2000_2020_USA`**  và **`wdi_2000_2020_JPN`** tương ứng với 3 quôc gia Việt Nam, Hoa Kỳ và Nhật Bản

In [ ]:
INDICATORS = {
    # Education
    'SE.PRM.UNER.FE':       'Children out of school, primary, female',
    'SE.PRM.UNER.MA':       'Children out of school, primary, male',
    'SE.XPD.TOTL.GD.ZS':    'Government expenditure on education, total (% of GDP)',
    'SE.XPD.PRIM.PC.ZS':    'Government expenditure per student, primary (% of GDP per capita)',
    'SE.XPD.SECO.PC.ZS':    'Government expenditure per student, secondary (% of GDP per capita)',
    'SE.XPD.TERT.PC.ZS':    'Government expenditure per student, tertiary (% of GDP per capita)',
    'SE.PRM.GINT.FE.ZS':    'Gross intake ratio in first grade of primary education, female (% of relevant age group)',
    'SE.PRM.GINT.MA.ZS':    'Gross intake ratio in first grade of primary education, male (% of relevant age group)',
    'SL.TLF.TOTL.FE.ZS':    'Labor force, female (% of total labor force)',
    'SL.TLF.TOTL.IN':       'Labor force, total',
    'SE.ADT.LITR.FE.ZS':    'Literacy rate, adult female (% of females ages 15 and above)',
    'SE.ADT.LITR.MA.ZS':    'Literacy rate, adult male (% of males ages 15 and above)',
    'SE.ADT.1524.LT.FE.ZS': 'Literacy rate, youth female (% of females ages 15-24)',
    'SE.ADT.1524.LT.MA.ZS': 'Literacy rate, youth male (% of males ages 15-24)',
    'SE.PRM.ENRL.TC.ZS':    'Pupil-teacher ratio, primary',
    'SE.PRM.REPT.FE.ZS':    'Repeaters, primary, female (% of female enrollment)',
    'SE.PRM.REPT.MA.ZS':    'Repeaters, primary, male (% of male enrollment)',
    'SE.PRE.ENRR':          'School enrollment, preprimary (% gross)',
    'SE.PRM.ENRR':          'School enrollment, primary (% gross)',
    'SE.SEC.ENRR':          'School enrollment, secondary (% gross)',
    'SE.PRM.TCAQ.ZS':       'Trained teachers in primary education (% of total teachers)',
    'SL.UEM.TOTL.FE.ZS':    'Unemployment, female (% of female labor force) (modeled ILO estimate)',
    'SL.UEM.TOTL.ZS':       'Unemployment, total (% of total labor force) (modeled ILO estimate)',

    # Climate Change
    'EG.ELC.ACCS.ZS':       'Access to electricity (% of population)',
    'AG.LND.IRIG.AG.ZS':    'Agricultural irrigated land (% of total agricultural land)',
    'AG.LND.AGRI.ZS':       'Agricultural land (% of land area)',
    'NV.AGR.TOTL.ZS':       'Agriculture, forestry, and fishing, value added (% of GDP)',
    'ER.H2O.FWTL.K3':       'Annual freshwater withdrawals, total (billion cubic meters)',
    'AG.LND.ARBL.ZS':       'Arable land (% of land area)',
    'AG.YLD.CREL.KG':       'Cereal yield (kg per hectare)',
    'EG.USE.ELEC.KH.PC':    'Electric power consumption (kWh per capita)',
    'EG.USE.PCAP.KG.OE':    'Energy use (kg of oil equivalent per capita)',
    'AG.LND.FRST.ZS':       'Forest area (% of land area)',
    'AG.LND.FRST.K2':       'Forest area (sq. km)',
    'AG.LND.EL5M.ZS':       'Land area where elevation is below 5 meters (% of total land area)',
    'SP.POP.GROW':          'Population growth (annual %)',
    'EN.POP.EL5M.ZS':       'Population living in areas where elevation is below 5 meters (% of total population)',
    'SP.POP.TOTL':          'Population, total',
    'EG.ELC.RNEW.ZS':       'Renewable electricity output (% of total electricity output)',
    'EG.FEC.RNEW.ZS':       'Renewable energy consumption (% of total final energy consumption)',
    'ER.PTD.TOTL.ZS':       'Terrestrial and marine protected areas (% of total territorial area)',
    'SP.URB.TOTL':          'Urban population',
    'SP.URB.TOTL.IN.ZS':    'Urban population (% of total population)',

    # Economic and Growth
    'NY.ADJ.SVNG.GN.ZS':    'Adjusted net savings, including particulate emission damage (% of GNI)',
    'GC.DOD.TOTL.GD.ZS':    'Central government debt, total (% of GDP)',
    'BM.GSR.ROYL.CD':       'Charges for the use of intellectual property, payments (BoP, current US$)',
    'BX.GSR.ROYL.CD':       'Charges for the use of intellectual property, receipts (BoP, current US$)',
    'BN.CAB.XOKA.CD':       'Current account balance (BoP, current US$)',
    'GC.XPN.TOTL.GD.ZS':    'Expense (% of GDP)',
    'NE.EXP.GNFS.ZS':       'Exports of goods and services (% of GDP)',
    'DT.DOD.DECT.GN.ZS':    'External debt stocks (% of GNI)',
    'DT.DOD.DECT.CD':       'External debt stocks, total (DOD, current US$)',
    'BX.KLT.DINV.CD.WD':    'Foreign direct investment, net inflows (BoP, current US$)',
    'NY.GDP.MKTP.CD':       'GDP (current US$)',
    'NY.GDP.MKTP.KD.ZG':    'GDP growth (annual %)',
    'NY.GDP.PCAP.CD':       'GDP per capita (current US$)',
    'NY.GDP.PCAP.KD.ZG':    'GDP per capita growth (annual %)',
    'NY.GDP.PCAP.PP.CD':    'GDP per capita, PPP (current international $)',
    'NY.GNP.PCAP.CD':       'GNI per capita, Atlas method (current US$)',
    'NY.GNP.PCAP.PP.CD':    'GNI per capita, PPP (current international $)',
    'NY.GNP.ATLS.CD':       'GNI, Atlas method (current US$)',
    'NY.GNP.MKTP.PP.CD':    'GNI, PPP (current international $)',
    'BX.GRT.EXTA.CD.WD':    'Grants, excluding technical cooperation (BoP, current US$)',
    'NE.GDI.TOTL.ZS':       'Gross capital formation (% of GDP)',
    'NY.GNS.ICTR.ZS':       'Gross savings (% of GDP)',
    'NE.IMP.GNFS.ZS':       'Imports of goods and services (% of GDP)',
    'NV.IND.TOTL.ZS':       'Industry (including construction), value added (% of GDP)',
    'NY.GDP.DEFL.KD.ZG':    'Inflation, GDP deflator (annual %)',
    'FP.CPI.TOTL.ZG':       'Inflation, consumer prices (annual %)',
    'NV.MNF.TECH.ZS.UN':    'Medium and high-tech manufacturing value added (% manufacturing value added)',
    'DT.ODA.ODAT.GN.ZS':    'Net ODA received (% of GNI)',
    'DT.ODA.ODAT.PC.ZS':    'Net ODA received per capita (current US$)',
    'DT.ODA.ODAT.CD':       'Net official development assistance received (current US$)',
    'PA.NUS.PPP':           'PPP conversion factor, GDP (LCU per international $)',
    'BX.TRF.PWKR.CD.DT':    'Personal remittances, received (current US$)',
    'PA.NUS.PRVT.PP':       'Price level ratio of PPP conversion factor (GDP) to market exchange rate',
    'GC.REV.XGRT.GD.ZS':    'Revenue, excluding grants (% of GDP)',
    'DT.DOD.DSTC.ZS':       'Short-term debt (% of total reserves)',
    'BX.GRT.TECH.CD.WD':    'Technical cooperation grants (BoP, current US$)',
    'DT.TDS.DECT.EX.ZS':    'Total debt service (% of exports of goods, services and primary income)',
    'FI.RES.TOTL.CD':       'Total reserves (includes gold, current US$)'
}
COUNTRIES = ['USA', 'JPN', 'VNM']
START_YEAR = 2008
END_YEAR = 2020
DATASETS = {}

for country in COUNTRIES:
    data = wb.data.DataFrame(
        series=list(INDICATORS.keys()),
        economy=country,
        time=range(START_YEAR, END_YEAR + 1),
    )
    data.to_csv(f'wdi_{START_YEAR}_{END_YEAR}_{country}.csv')
    DATASETS[country] = data.reset_index()

DATASETS['USA']

## **Tiền xử lý dữ liệu**

### **Định dạng lại bộ dữ liệu**

**Mục đích**: Chuyển dữ liệu từ dạng rộng (wide format) sang dạng bảng chuẩn, giúp dễ dàng phân tích theo năm và quốc gia.

**Các thực hiện**:

1. Đọc dữ liệu với mỗi chỉ số (*series*) là một hàng, các năm (*YR2000–YR2020*) là cột vào cột Year và giá trị vào cột Value.

2. Chuyển sang long format bằng cách dùng melt() để đưa năm vào cột Year và giá trị vào cột Value.

3. Chuẩn hóa dữ liệu bằng cách chuyển Year về dạng số, thay dấu chấm (.) trong tên chỉ số bằng gạch dưới (_). 
    
    Ví dụ: **AG.LND.ARBL.ZS** $\rightarrow$ **AG_LND_ARBL_ZS**.

4. Pivot lại dữ liệu bằng cách chuyển mỗi chỉ số thành một cột riêng, với Year làm cột chính.

5. Lưu kết quả sau khi đã áp dụng cho từng quốc gia và ghi đè vào file tương ứng.

In [ ]:
def transform_data(df: pd.DataFrame, country: str) -> None:
    df_melted = df.melt(id_vars=['series'], var_name='year', value_name='value')
    df_melted['year'] = df_melted['year'].str[-4:]
    
    df_pivoted = df_melted.pivot(index='year', columns='series', values='value')
    df_pivoted.columns.name = None
    df_pivoted.columns = df_pivoted.columns.str.replace('.', '_')
    df_pivoted.to_csv(path_or_buf=f'wdi_{START_YEAR}_{END_YEAR}_{country}.csv')
    
    print(f'Đã biến đổi và lưu dữ liệu của {country} thành công!')
    
for country in COUNTRIES:
    transform_data(df=DATASETS[country], country=country)

## **Đọc bộ dữ liệu**

In [ ]:
vn = pd.read_csv(filepath_or_buffer=f'./wdi_{START_YEAR}_{END_YEAR}_VNM.csv')
us = pd.read_csv(filepath_or_buffer=f'./wdi_{START_YEAR}_{END_YEAR}_USA.csv')
jp = pd.read_csv(filepath_or_buffer=f'./wdi_{START_YEAR}_{END_YEAR}_JPN.csv')

In [ ]:
vn.head()

In [ ]:
us.head()

In [ ]:
jp.head()

In [ ]:
features = [
    "SE_PRM_UNER_FE",        # Trẻ em nữ không đi học ở cấp tiểu học
    "SE_PRM_UNER_MA",        # Trẻ em nam không đi học, cấp tiểu học
    "SE_XPD_TOTL_GD_ZS",     # Chi tiêu công cho giáo dục (% GDP)
    "SE_XPD_PRIM_PC_ZS",     # Chi tiêu công trên mỗi học sinh tiểu học (% GDP bình quân đầu người)
    "SE_XPD_SECO_PC_ZS",     # Chi tiêu công trên mỗi học sinh trung học (% GDP bình quân đầu người)
    "SE_XPD_TERT_PC_ZS",     # Chi tiêu công trên mỗi sinh viên đại học (% GDP bình quân đầu người)
    "SE_PRM_GINT_FE_ZS",     # Tỷ lệ tuyển sinh lớp 1, nữ
    "SE_PRM_GINT_MA_ZS",     # Tỷ lệ tuyển sinh lớp 1, nam
    "SL_TLF_TOTL_FE_ZS",     # Lực lượng lao động nữ (% tổng lực lượng lao động)
    "SL_TLF_TOTL_IN",        # Tổng lực lượng lao động
]

features_2 =[
    "SE_ADT_LITR_FE_ZS",     # Tỷ lệ biết chữ ở người lớn, nữ
    "SE_ADT_LITR_MA_ZS",     # Tỷ lệ biết chữ ở người lớn, nam
    "SE_ADT_1524_LT_FE_ZS",  # Tỷ lệ biết chữ ở thanh niên, nữ
    "SE_ADT_1524_LT_MA_ZS",  # Tỷ lệ biết chữ ở thanh niên, nam
    "SE_PRM_ENRL_TC_ZS",     # Tỷ lệ học sinh trên giáo viên cấp tiểu học
    "SE_PRM_REPT_FE_ZS",     # Tỷ lệ học sinh nữ lưu ban cấp tiểu học
    "SE_PRM_REPT_MA_ZS",     # Tỷ lệ học sinh nam lưu ban cấp tiểu học
    "SE_PRE_ENRR",           # Tỷ lệ nhập học mầm non (% tổng số)
    "SE_PRM_ENRR",           # Tỷ lệ nhập học tiểu học (% tổng số)
    "SE_SEC_ENRR",           # Tỷ lệ nhập học trung học (% số lượng thực tế)
    "SE_PRM_TCAQ_ZS",        # Tỷ lệ giáo viên tiểu học được đào tạo
    "SL_UEM_TOTL_FE_ZS",     # Tỷ lệ thất nghiệp ở nữ
    "SL_UEM_TOTL_ZS",        # Tổng tỷ lệ thất nghiệp
]

In [ ]:
jp['NV_IND_TOTL_ZS']

## **Khám phá dữ liệu**

Bảng bên dưới mô tả chi tiết ý nghĩa của các thuộc tính của World Developemtn Indicators trong bộ dữ liệu **`wdi_2000_2020_VNM.csv`**, **`wdi_2000_2020_USA.csv`** và **`wdi_2000_2020_JPN.csv`**

##### **Các chỉ số thuộc về lĩnh vực Giáo dục (Education)**

<center>

| **Mã chỉ số** | **Tên chỉ số** | **Mô tả** |
| ---- | ---- | ---- |
| **SE.PRM.UNER.FE** | Trẻ em nữ không đi học ở cấp tiểu học *[Children out of school, primary, female]* | Số lượng trẻ em gái trong độ tuổi đi học tiểu học nhưng không đi học |
| **SE.PRM.UNER.MA** | Trẻ em nam không đi học, cấp tiểu học *[Children out of school, primary, male]* | Số lượng trẻ em trai trong độ tuổi đi học tiểu học nhưng không đi học |
| **SE.XPD.TOTL.GD.ZS** |Chi tiêu công cho giáo dục (% GDP) *[Government expenditure on education, total (% of GDP)]*	| Tổng chi tiêu của chính phủ cho giáo dục tính theo phần trăm GDP |
| **SE.XPD.PRIM.PC.ZS**	| Chi tiêu công trên mỗi học sinh tiểu học (% GDP bình quân đầu người) *[Government expenditure per student, primary (% of GDP per capita)]* | Tỷ lệ chi tiêu công cho mỗi học sinh cấp tiểu học so với GDP bình quân đầu người |
| **SE.XPD.SECO.PC.ZS**	| Chi tiêu công trên mỗi học sinh trung học (% GDP bình quân đầu người) *[Government expenditure per student, secondary (% of GDP per capita)]* | Tỷ lệ chi tiêu công cho mỗi học sinh cấp trung học so với GDP bình quân đầu người |
| **SE.XPD.TERT.PC.ZS** | Chi tiêu công trên mỗi sinh viên đại học (% GDP bình quân đầu người) *[Government expenditure per student, tertiary (% of GDP per capita)]* |	Tỷ lệ chi tiêu công cho mỗi sinh viên đại học so với GDP bình quân đầu người |
| **SE.PRM.GINT.FE.ZS**	| Tỷ lệ tuyển sinh lớp 1, nữ *[Gross intake ratio in first grade of primary education, female (% of relevant age group)]* | Số lượng học sinh nữ nhập học lớp 1 trên tổng số trẻ em nữ trong độ tuổi tương ứng (tính theo phần trăm) |
| **SE.PRM.GINT.MA.ZS** | Tỷ lệ tuyển sinh lớp 1, nam *[Gross intake ratio in first grade of primary education, male (% of relevant age group)]* | Số lượng học sinh nam nhập học lớp 1 trên tổng số trẻ em nam trong độ tuổi tương ứng (tính theo phần trăm) |
| **SL.TLF.TOTL.FE.ZS** | Lực lượng lao động nữ (% tổng lực lượng lao động) *[Labor force, female (% of total labor force)]* | Tỷ lệ lao động nữ trong tổng lực lượng lao động của quốc gia |
| **SL.TLF.TOTL.IN** | Tổng lực lượng lao động *[Labor force, total]* | Tổng số người tham gia vào lực lượng lao động của quốc gia, bao gồm cả người có việc làm và người thất nghiệp đang tìm việc |
| **SE.ADT.LITR.FE.ZS** | Tỷ lệ biết chữ ở người lớn, nữ *[Literacy rate, adult female (% of females ages 15 and above)]* |	Tỷ lệ phần trăm phụ nữ từ 15 tuổi trở lên có thể đọc và viết |
| **SE.ADT.LITR.MA.ZS**	| Tỷ lệ biết chữ ở người lớn, nam *[Literacy rate, adult male (% of males ages 15 and above)]* | Tỷ lệ phần trăm nam giới từ 15 tuổi trở lên có thể đọc và viết |
| **SE.ADT.1524.LT.FE.ZS** |	Tỷ lệ biết chữ ở thanh niên, nữ *[Literacy rate, youth female (% of females ages 15-24)]* | Tỷ lệ phần trăm nữ từ 15-24 tuổi có thể đọc và viết |
| **SE.ADT.1524.LT.MA.ZS** |	Tỷ lệ biết chữ ở thanh niên, nam *[Literacy rate, youth male (% of males ages 15-24)]* | Tỷ lệ phần trăm nam từ 15-24 tuổi có thể đọc và viết |
| **SE.PRM.ENRL.TC.ZS**	| Tỷ lệ học sinh trên giáo viên cấp tiểu học *[Pupil-teacher ratio, primary]* |	Số lượng học sinh trung bình trên một giáo viên cấp tiểu học |
| **SE.PRM.REPT.FE.ZS** | Tỷ lệ học sinh nữ lưu ban cấp tiểu học *[Repeaters, primary, female (% of female enrollment)]* | Phần trăm học sinh nữ bị lưu ban tại cấp tiểu học |
| **SE.PRM.REPT.MA.ZS**	| Tỷ lệ học sinh nam lưu ban cấp tiểu học *[Repeaters, primary, male (% of male enrollment)]* |	Phần trăm học sinh nam bị lưu ban tại cấp tiểu học |
| **SE.PRE.ENRR** | Tỷ lệ nhập học mầm non (% tổng số) *[School enrollment, preprimary (% gross)]* | Tỷ lệ tổng số trẻ em nhập học mầm non so với tổng số trẻ em trong độ tuổi mầm non |
| **SE.PRM.ENRR** | Tỷ lệ nhập học tiểu học (% tổng số) *[School enrollment, primary (% gross)]* | Tỷ lệ tổng số học sinh nhập học cấp tiểu học so với tổng số trẻ em trong độ tuổi tiểu học |
| **SE.SEC.ENRR** |	Tỷ lệ nhập học trung học (% số lượng thực tế) *[School enrollment, secondary (% gross)]* | Tỷ lệ tổng số học sinh nhập học trung học so với số trẻ em trong độ tuổi tương ứng |
| **SE.PRM.TCAQ.ZS** |	Tỷ lệ giáo viên tiểu học được đào tạo *[Trained teachers in primary education (% of total teachers)]* | Phần trăm giáo viên tiểu học có bằng cấp đào tạo chính thức |
| **SL.UEM.TOTL.FE.ZS** | Tỷ lệ thất nghiệp ở nữ *[Unemployment, female (% of female labor force) (modeled ILO estimate)]* | Phần trăm nữ giới trong lực lượng lao động không có việc làm nhưng đang tìm kiếm việc làm |
| **SL.UEM.TOTL.ZS** | Tổng tỷ lệ thất nghiệp *[Unemployment, total (% of total labor force) (modeled ILO estimate)]* |	Phần trăm tổng trong lực lượng lao động không có việc làm nhưng đang tìm kiếm việc làm |

</center>

##### **Các chỉ số thuộc về lĩnh vực Climate Change (Biến đổi khí hậu)**

| **Mã chỉ số** | **Tên chỉ số** | **Mô tả** |
| ---- | ---- | ---- |
| **EG.ELC.ACCS.ZS** | Dùng điện năng (% dân số) *[Access to electricity (% of population)]* | 	Tỷ lệ phần trăm dân số có quyền truy cập vào điện |
| **AG.LND.IRIG.AG.ZS** | Đất nông nghiệp có tưới tiêu (% tổng diện tích đất nông nghiệp)	 *[Agricultural irrigated land (% of total agricultural land)]* | Diện tích đất nông nghiệp được tưới tiêu trên tổng diện tích đất nông nghiệp |
| **AG.LND.AGRI.ZS** | Đất nông nghiệp (% tổng diện tích đất) *[Agricultural land (% of land area)]* | Phần trăm diện tích đất được sử dụng cho mục đích nông nghiệp so với tổng diện tích đất |
| **NV.AGR.TOTL.ZS** | Giá trị gia tăng của nông, lâm, ngư nghiệp (% GDP) *[Agriculture, forestry, and fishing, value added (% of GDP)]* | Tỷ lệ đóng góp của nông nghiệp, lâm nghiệp và thủy sản vào tổng sản phẩm quốc nội (GDP) |
| **ER.H2O.FWTL.K3** | Tổng lượng nước ngọt dùng hàng năm (tỷ m³) *[ Annual freshwater withdrawals, total (billion cubic meters)]* | Lượng nước ngọt được sử dụng hàng năm cho mục đích sinh hoạt, công nghiệp và nông nghiệp |
| **AG.LND.ARBL.ZS** | Đất canh tác (% tổng diện tích đất) *[Arable land (% of land area)]* | Tỷ lệ phần trăm đất có thể trồng trọt trên tổng diện tích đất |
| **AG.YLD.CREL.KG** | Sản lượng ngũ cốc (kg/ha)	*[Cereal yield (kg per hectare)]* | Sản lượng ngũ cốc trung bình trên mỗi hecta đất trồng trọt |
| **EG.USE.ELEC.KH.PC** | Tiêu thụ điện bình quân đầu người (kWh/người) *[Electric power consumption (kWh per capita)]* | Lượng điện năng tiêu thụ trung bình mỗi người trong một năm |
| **EG.USE.PCAP.KG.OE** | Tiêu thụ năng lượng bình quân đầu người (kg dầu tương đương/người) *[Energy use (kg of oil equivalent per capita)]* | Lượng năng lượng trung bình mỗi người tiêu thụ hàng năm, tính theo đơn vị kg dầu tương đương |
| **AG.LND.FRST.ZS** | Diện tích rừng (% tổng diện tích đất) *[Forest area (% of land area)]* | Tỷ lệ diện tích rừng so với tổng diện tích đất của quốc gia |
| **AG.LND.FRST.K2** | Tổng diện tích rừng (km²)  *[Forest area (sq. km)]* | Tổng diện tích đất được bao phủ bởi rừng, tính theo đơn vị km² |
| **AG.LND.EL5M.ZS** | Đất thấp dưới 5 mét so với mực nước biển (% tổng diện tích đất) | *[Land area where elevation is below 5 meters (% of total land area)]* | Phần trăm tổng diện tích đất nằm ở độ cao dưới 5 mét so với mực nước biển |
| **SP.POP.GROW** |  Tốc độ tăng dân số (% hàng năm) *[Population growth (annual %)]* | Tốc độ thay đổi dân số theo tỷ lệ phần trăm hàng năm |
| **EN.POP.EL5M.ZS** | Dân số sống ở vùng thấp dưới 5 mét so với mực nước biển (% tổng dân số) *[Population living in areas where elevation is below 5 meters (% of total population)]* | Tỷ lệ phần trăm dân số sinh sống ở khu vực có độ cao dưới 5 mét so với mực nước biển |
| **SP.POP.TOTL** | Tổng dân số *[Population, total]* | Tổng số người sinh sống trong một quốc gia hoặc vùng lãnh thổ tại một thời điểm nhất định |
| **EG.ELC.RNEW.ZS** | Sản lượng điện tái tạo (% tổng sản lượng điện) *[Renewable electricity output (% of total electricity output)]* | Tỷ lệ điện sản xuất từ các nguồn năng lượng tái tạo (như gió, mặt trời, thủy điện) so với tổng lượng điện sản xuất |
| **EG.FEC.RNEW.ZS** | Tiêu thụ năng lượng tái tạo (% tổng tiêu thụ năng lượng cuối cùng) *[Renewable energy consumption (% of total final energy consumption)]* | Tỷ lệ năng lượng tái tạo trong tổng mức tiêu thụ năng lượng cuối cùng | 
| **ER.PTD.TOTL.ZS** | Khu vực bảo tồn trên cạn và trên biển (% tổng diện tích lãnh thổ) *[Terrestrial and marine protected areas (% of total territorial area)]* | Phần trăm diện tích lãnh thổ được bảo vệ trên cạn và biển, bao gồm các khu bảo tồn thiên nhiên, vườn quốc gia |
| **SP.URB.TOTL** | Dân số đô thị *[Urban population]* | Tổng số người sinh sống tại khu vực đô thị |
| **SP.URB.TOTL.IN.ZS** | Dân số đô thị (% tổng dân số) *[Urban population (% of total population)]* | Phần trăm dân số sống tại khu vực đô thị so với tổng dân số của quốc gia |


##### **Các chỉ số thuộc về lĩnh vực Economic and Growth (Phát triển kinh tế)**

| Mã chỉ số | Tên chỉ số | Mô tả |
| ----  | ----- | ---- |
| **NY.ADJ.SVNG.GN.ZS** | Tiết kiệm ròng điều chỉnh, bao gồm thiệt hại do phát thải hạt (% GNI) *[Adjusted net savings, including particulate emission damage (% of GNI)]* | Chỉ số đo lường mức tiết kiệm ròng sau khi điều chỉnh cho suy giảm tài nguyên thiên nhiên và thiệt hại do ô nhiễm không khí |
| **NV.AGR.TOTL.ZS** | Giá trị gia tăng của nông, lâm, ngư nghiệp (% GDP) *[Agriculture, forestry, and fishing, value added (% of GDP)]* | Đóng góp của ngành nông nghiệp, lâm nghiệp và thủy sản vào GDP |
| **GC.DOD.TOTL.GD.ZS** | Nợ chính phủ trung ương (% GDP) *[Central government debt, total (% of GDP)]* | Tổng nợ của chính phủ trung ương so với GDP của quốc gia |
| **BM.GSR.ROYL.CD** | Thanh toán sử dụng quyền sở hữu trí tuệ (BoP, Tính theo tỷ giá đồng Dollar hiện tại) *[Charges for the use of intellectual property, payments (BoP, current US$)]* | Tổng số tiền một quốc gia thanh toán cho việc sử dụng bằng sáng chế, bản quyền, thương hiệu và các tài sản trí tuệ khác |
| **BX.GSR.ROYL.CD** | Thu nhập từ quyền sở hữu trí tuệ (BoP, Tính theo tỷ giá đồng Dollar hiện tại) *[Charges for the use of intellectual property, receipts (BoP, current US$)]* | Tổng số tiền một quốc gia nhận được từ việc cấp phép sử dụng tài sản trí tuệ |
| **BN.CAB.XOKA.CD** | Cán cân tài khoản vãng lai (BoP, Tính theo tỷ giá đồng Dollar hiện tại) *[Current account balance (BoP, current US$)]* | Sự chênh lệch giữa xuất nhập khẩu hàng hóa, dịch vụ, thu nhập ròng và chuyển nhượng ròng |
| **GC.XPN.TOTL.GD.ZS** |  Chi tiêu của chính phủ (% GDP) *[Expense (% of GDP)]* | Tổng chi tiêu của chính phủ so với GDP |
| **NE.EXP.GNFS.ZS** | Xuất khẩu hàng hóa và dịch vụ (% GDP) *[Exports of goods and services (% of GDP)]* | Tổng giá trị xuất khẩu hàng hóa và dịch vụ so với GDP |
| **DT.DOD.DECT.GN.ZS** | Tổng nợ nước ngoài (% GNI) *[External debt stocks (% of GNI)]* |Tổng giá trị nợ nước ngoài của một quốc gia so với tổng thu nhập quốc dân |
| **DT.DOD.DECT.CD** | Tổng nợ nước ngoài (Tính theo tỷ giá đồng Dollar hiện tại) **[External debt stocks, total (DOD, current US$)]** | Tổng giá trị nợ nước ngoài của quốc gia, tính bằng đô la Mỹ |
| **BX.KLT.DINV.CD.WD** | Đầu tư trực tiếp nước ngoài, dòng vốn vào ròng (BoP, Tính theo tỷ giá đồng Dollar hiện tại) *[Foreign direct investment, net inflows (BoP, current US$)]* | Tổng số vốn đầu tư trực tiếp từ nước ngoài vào quốc gia |
| **NY.GDP.MKTP.CD** | GDP (Tính theo tỷ giá đồng Dollar hiện tại) *[GDP (current US$)]* | Tổng sản phẩm quốc nội (GDP) của một quốc gia theo giá trị thị trường hiện tại |
| **NY.GDP.MKTP.KD.ZG** | Tăng trưởng GDP (% hàng năm) *[GDP growth (annual %)]* | Tỷ lệ tăng trưởng hàng năm của GDP, điều chỉnh theo lạm phát |
| **NY.GDP.PCAP.CD** | GDP bình quân đầu người (Tính theo tỷ giá đồng Dollar hiện tại) *[GDP per capita (current US$)]* | Tổng GDP chia cho dân số của quốc gia |
| **NY.GDP.PCAP.KD.ZG** | Tăng trưởng GDP bình quân đầu người (% hàng năm) *[GDP per capita growth (annual %)]* | Tốc độ tăng trưởng của GDP bình quân đầu người theo giá thực tế |
| **NY.GDP.PCAP.PP.CD** | GDP bình quân đầu người theo sức mua tương đương (PPP, USD quốc tế) *[GDP per capita, PPP (current international $)]* | GDP bình quân đầu người được điều chỉnh theo ngang giá sức mua (PPP) |
| **NY.GNP.PCAP.CD** | GNI bình quân đầu người, phương pháp Atlas (Tính theo tỷ giá đồng Dollar hiện tại) *[GNI per capita, Atlas method (current US$)]* | Tổng thu nhập quốc dân bình quân đầu người theo phương pháp Atlas |
| **NY.GNP.PCAP.PP.CD** | GNI bình quân đầu người theo PPP (USD quốc tế) *[GNI per capita, PPP (current international $)]* | Tổng thu nhập quốc dân bình quân đầu người điều chỉnh theo ngang giá sức mua |
| **NY.GNP.ATLS.CD** | GNI, phương pháp Atlas (Tính theo tỷ giá đồng Dollar hiện tại) *[GNI, Atlas method (current US$)]* | Tổng thu nhập quốc dân của quốc gia theo phương pháp Atlas |
| **NY.GNP.MKTP.PP.CD** | GNI theo PPP (USD quốc tế) *[GNI, PPP (current international $)]* |	Tổng thu nhập quốc dân được điều chỉnh theo ngang giá sức mua |
| **BX.GRT.EXTA.CD.WD** | Viện trợ không hoàn lại, trừ hợp tác kỹ thuật (BoP, Tính theo tỷ giá đồng Dollar hiện tại) *[Grants, excluding technical cooperation (BoP, current US$)]*	| Tổng giá trị viện trợ không hoàn lại mà một quốc gia nhận được, không bao gồm hợp tác kỹ thuật |
| **NE.GDI.TOTL.ZS** | Tổng đầu tư vốn (% GDP) *[Gross capital formation (% of GDP)]* | Tổng chi tiêu cho đầu tư vốn so với GDP |
| **NY.GNS.ICTR.ZS** | Tiết kiệm gộp (% GDP) *[Gross savings (% of GDP)]* | Phần trăm GDP được tiết kiệm sau khi trừ đi tiêu dùng của chính phủ và hộ gia đình |
| **NE.IMP.GNFS.ZS** | Nhập khẩu hàng hóa và dịch vụ (% GDP) *[Imports of goods and services (% of GDP)]* | Tổng giá trị nhập khẩu hàng hóa và dịch vụ so với GDP |
| **NV.IND.TOTL.ZS** | Giá trị gia tăng của ngành công nghiệp (% GDP) *[Industry (including construction), value added (% of GDP)]* | Đóng góp của ngành công nghiệp, bao gồm xây dựng, vào GDP |
| **NY.GDP.DEFL.KD.ZG** | Lạm phát, chỉ số giảm phát GDP (% hàng năm) *[Inflation, GDP deflator (annual %)]* | Phần trăm thay đổi lạm phát hàng năm |
| **FP.CPI.TOTL.ZG** | Lạm phát, chỉ số giá tiêu dùng (% hàng năm) *[Inflation, consumer prices (annual %)]* | Tỷ lệ lạm phát hàng năm dựa trên chỉ số giá tiêu dùng (CPI) |
| **NV.MNF.TECH.ZS.UN** | Giá trị gia tăng của ngành sản xuất công nghệ vừa và cao (% tổng giá trị gia tăng của ngành sản xuất) *[Medium and high-tech manufacturing value added (% manufacturing value added)]* | Phần trăm giá trị gia tăng của ngành công nghiệp chế tạo có công nghệ vừa và cao |
| **DT.ODA.ODAT.GN.ZS** | Hỗ trợ phát triển nhận được (% GNI) *[Net ODA received (% of GNI)]* | Tổng viện trợ phát triển chính thức mà quốc gia nhận được so với GNI |
| **DT.ODA.ODAT.PC.ZS** | Hỗ trợ phát triển nhận được trên đầu người (Tính theo tỷ giá đồng Dollar hiện tại) *[Net ODA received per capita (current US$)]* |  Tổng viện trợ phát triển chính thức mà quốc gia nhận được chia cho dân số  |
| **DT.ODA.ODAT.CD** | Hỗ trợ phát triển nhận được (Tính theo tỷ giá đồng Dollar hiện tại) *[Net official development assistance received (current US$)]* | Tổng giá trị viện trợ phát triển chính thức mà quốc gia nhận được | 
| **PA.NUS.PPP** | Hệ số chuyển đổi PPP, GDP (LCU trên 1 USD quốc tế) *[PPP conversion factor, GDP (LCU per international $)]* | Hệ số chuyển đổi từ đơn vị tiền tệ nội địa (LCU) sang đô la quốc tế theo phương pháp ngang giá sức mua (PPP) |
| **BX.TRF.PWKR.CD.DT** | Kiều hối cá nhân nhận được (Tính theo tỷ giá đồng Dollar hiện tại) *[Personal remittances, received (current US$)]* | Tổng giá trị kiều hối cá nhân mà một quốc gia nhận được từ công dân làm việc ở nước ngoài |
| **PA.NUS.PRVT.PP** | Tỷ lệ mức giá của hệ số chuyển đổi PPP (GDP) so với tỷ giá hối đoái thị trường *[Price level ratio of PPP conversion factor (GDP) to market exchange rate]* | Tỷ lệ giữa hệ số chuyển đổi PPP và tỷ giá hối đoái thị trường, đo lường mức giá nội địa so với giá quốc tế |
| **GC.REV.XGRT.GD.ZS** | Doanh thu, không bao gồm viện trợ (% GDP) *[Revenue, excluding grants (% of GDP)]* | Tổng doanh thu của chính phủ không bao gồm viện trợ quốc tế, tính theo % GDP | 
| **DT.DOD.DSTC.ZS** | Nợ ngắn hạn (% tổng dự trữ) *[Short-term debt (% of total reserves)]* | Tổng nợ ngắn hạn của một quốc gia so với tổng dự trữ ngoại hối và vàng |
| **BX.GRT.TECH.CD.WD** | Viện trợ hợp tác kỹ thuật (BoP, Tính theo tỷ giá đồng Dollar hiện tại) *[Technical cooperation grants (BoP, current US$)]* | Tổng giá trị viện trợ quốc tế dành cho hợp tác kỹ thuật mà một quốc gia nhận được | 
| **DT.TDS.DECT.EX.ZS** | Nghĩa vụ trả nợ (% xuất khẩu hàng hóa, dịch vụ và thu nhập sơ cấp) *[Total debt service (% of exports of goods, services and primary income)]* | Tổng nghĩa vụ trả nợ (cả gốc và lãi) của một quốc gia so với tổng giá trị xuất khẩu hàng hóa, dịch vụ và thu nhập từ nước ngoài |
| **FI.RES.TOTL.CD** | Dự trữ ngoại hối và vàng (Tính theo tỷ giá đồng Dollar hiện tại) *[Total reserves (includes gold, current US$)]* | Tổng giá trị dự trữ ngoại tệ và vàng của quốc gia |


### **Tổng quan về dữ liệu**

**Bộ dữ liệu của Việt Nam *[wdi_2000_2020_VNM]***

In [ ]:
vn.info()

In [ ]:
vn_null_columns = vn.columns[vn.isnull().all()].tolist()

print('Các thuộc tính trong bộ dữ liệu Việt Nam [wdi_2000_2020_VNM] có toàn bộ giá trị là null:')
print(vn_null_columns)

**Nhận xét về bộ dữ liệu của Việt Nam *[wdi_2000_2020_VNM]* sau khi xử lý**
- Dữ liệu còn *21* dòng và *82* thuộc tính (cột).
- Tất cả các thuộc tính đều có kiểu dữ liệu là *float64* trừ thuộc tính **Year** có kiểu dữ liệu là *int32*.
- Các thuộc tính không có dữ liệu nào bao gồm
    - **AG_LND_IRIG_AG_ZS**
    - **BM_GSR_ROYL_CD**
    - **BX_GSR_ROYL_CD**
    - **GC_DOD_TOTL_GD_ZS**
    - **GC_REV_XGRT_GD_ZS**
    - **GC_XPN_TOTL_GD_ZS**
    - **SE_PRM_UNER_FE**
    - **SE_PRM_UNER_MA**
    - **SE_XPD_SECO_PC_ZS**

**Bộ dữ liệu của Hoa Kỳ *[wdi_2000_2020_USA]***

In [ ]:
us.info()

In [ ]:
us_null_columns = us.columns[us.isnull().all()].tolist()

print('Các thuộc tính trong bộ dữ liệu Hoa Kỳ [wdi_2000_2020_USA] có toàn bộ giá trị là null:')
print(us_null_columns)

**Nhận xét về bộ dữ liệu của Hoa Kỳ *[wdi_2000_2020_USA]* sau khi xử lý**
- Dữ liệu còn *21* dòng và *82* thuộc tính (cột).
- Tất cả các thuộc tính đều có kiểu dữ liệu là *float64* trừ thuộc tính **Year** có kiểu dữ liệu là *int32*.
- Các thuộc tính không có dữ liệu nào bao gồm 
    - **BX_GRT_EXTA_CD_WD**
    - **BX_GRT_TECH_CD_WD**
    - **DT_DOD_DECT_CD**
    - **DT_DOD_DECT_GN_ZS**
    - **DT_DOD_DSTC_ZS**
    - **DT_ODA_ODAT_CD**
    - **DT_ODA_ODAT_GN_ZS**
    - **DT_ODA_ODAT_PC_ZS**
    - **DT_TDS_DECT_EX_ZS**
    - **SE_ADT_1524_LT_FE_ZS**
    - **SE_ADT_1524_LT_MA_ZS**
    - **SE_ADT_LITR_FE_ZS**
    - **SE_ADT_LITR_MA_ZS**

**Bộ dữ liệu của Nhật Bản *[wdi_2000_2020_JPN]***

In [ ]:
jp.info()

In [ ]:
jp_null_columns = jp.columns[jp.isnull().all()].tolist()

print('Các thuộc tính trong bộ dữ liệu Nhật Bản [wdi_2000_2020_JPN] có toàn bộ giá trị là null:')
print(jp_null_columns)

**Nhận xét về bộ dữ liệu của Nhật Bản *[wdi_2000_2020_JPN]* sau khi xử lý**
- Dữ liệu còn *21* dòng và *75* thuộc tính (cột).
- Tất cả các thuộc tính đều có kiểu dữ liệu là *float64* trừ thuộc tính **Year** có kiểu dữ liệu là *int32*.
- Các thuộc tính không có dữ liệu nào bao gồm 
    - **BX_GRT_EXTA_CD_WD**
    - **BX_GRT_TECH_CD_WD**
    - **DT_DOD_DECT_CD**
    - **DT_DOD_DECT_GN_ZS**
    - **DT_DOD_DSTC_ZS**
    - **DT_ODA_ODAT_CD**
    - **DT_ODA_ODAT_GN_ZS**
    - **DT_ODA_ODAT_PC_ZS**
    - **DT_TDS_DECT_EX_ZS**
    - **GC_REV_XGRT_GD_ZS**
    - **SE_ADT_1524_LT_FE_ZS**
    - **SE_ADT_1524_LT_MA_ZS**
    - **SE_ADT_LITR_FE_ZS**
    - **SE_ADT_LITR_MA_ZS**
    - **SE_PRM_GINT_FE_ZS**
    - **SE_PRM_GINT_MA_ZS**
    - **SE_PRM_TCAQ_ZS**
    - **SE_PRM_UNER_FE**
    - **SE_PRM_UNER_MA**

## **Các câu hỏi**

### **Chủ đề Giáo dục**

#### **Câu 1**: Chi tiêu của chính phủ cho giáo dục (tổng cộng, tính bằng % GDP) tại Nhật Bản, Việt Nam và Hoa Kỳ đã thay đổi như thế nào từ năm 2000 đến năm 2020?


**Thuộc tính được sử dụng**
1. *SE_XPD_TOTL_GD_ZS* - Chi tiêu của chính phủ cho giáo dục, tổng cộng (% GDP).
2. *Year* - Năm từ 2000 đến 2020.

**Các bước thực hiện**
1. Lọc dữ liệu (lấy thuộc tính *Year* và *SE_XPD_TOTL_GD_ZS*) từ bộ dữ liệu của Nhật Bản, Việt Nam và Hoa Kỳ.
2. Vẽ biểu đồ đường để thể hiện sự thay đổi của chi tiêu giáo dục theo thời gian cho từng quốc gia nhằm so sánh mức chi tiêu giáo dục (% GDP) qua các năm cho từng quốc gia và xác định xu hướng (tăng, giảm hoặc dao động) từ 2000 đến 2020.
3. Tùy chỉnh trục, nhãn và tiêu đề cho biểu đồ.
4. Lưu biểu đồ dưới định dạng SVG và hiển thị kết quả.
5. Viết nhận xét.

In [ ]:
years = np.arange(2008, 2021)

plt.figure(figsize=(12, 6))
plt.plot(years, jp['SE_XPD_TOTL_GD_ZS'], 'b-', label='Nhật Bản')
plt.plot(years, vn['SE_XPD_TOTL_GD_ZS'], 'r-', label='Việt Nam')
plt.plot(years, us['SE_XPD_TOTL_GD_ZS'], 'k-', label='Hoa Kỳ')

plt.xticks(years)
plt.title('Chi tiêu của chính phủ cho giáo dục (tổng cộng, tính bằng % GDP) tại Nhật Bản, Việt Nam và Hoa Kỳ (2000-2020)')
plt.xlabel('Năm')
plt.ylabel('Chi tiêu của chính phủ cho giáo dục (% of GDP)')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

**Nhận xét về chi tiêu của chính phủ cho giáo dục (tổng cộng, tính bằng % GDP) tại Nhật Bản, Việt Nam và Hoa Kỳ (2000-2020)**

- Ở **Nhật Bản (đường màu xanh)**, chi tiêu cho giáo dục của chính phủ dao động trong khoảng **3.1% - 3.6% GDP**, có xu hướng tổng thể **ổn định nhưng hơi giảm nhẹ**. Mức chi tiêu cao nhất là **khoảng 3.6% vào năm 2010**, trong khi mức thấp nhất là **khoảng 3.1% vào năm 2018**

- Ở **Việt Nam (đường màu đỏ)**, chi tiêu cho giáo dục dao động khá mạnh trong khoảng **3.0% - 4.9% GDP**, với xu hướng tổng thể **giảm dần theo thời gian**. Mức chi tiêu cao nhất đạt **gần 4.9% vào năm 2008**, nhưng sau đó giảm dần và đạt mức thấp nhất **khoảng 3.0% vào năm 2019**

- Ở **Hoa Kỳ (đường màu đen)**, chi tiêu cho giáo dục biến động trong khoảng **4.8% - 6.7% GDP**, có xu hướng tổng thể **giảm dần**. Mức cao nhất là khoảng **6.7% vào năm 2009**, sau đó giảm đáng kể xuống **4.8% vào năm 2015**, rồi phục hồi nhẹ vào năm 2020

**Kết luận**
- Nhật Bản có mức chi tiêu **ổn định** nhưng **thấp hơn** so với hai nước còn lại.

- Việt Nam có sự **sụt giảm** rõ rệt sau năm 2010. Điều này phản ánh sự thay đổi chính sách tài chính cho giáo dục.

- Hoa Kỳ có mức chi tiêu **cao nhất** trong ba nước nhưng có xu hướng **giảm mạnh** từ sau năm 2010.

#### **Câu 2:** Mối quan hệ giữa lực lượng lao động và tăng trưởng GDP hằng năm ở Việt Nam trong giai đoạn 2000-2020 là như thế nào? Liệu rằng khi lực lượng lao động tăng lên, GDP có xu hướng tăng trưởng theo không?


**Thuộc tính được sử dụng**
1. *NY_GDP_MKTP_KD_ZG* - Tăng trưởng GDP hàng năm (%).

2. *SL_TLF_TOTL_IN* - Lực lượng lao động.


**Các bước thực hiện**
1. Lấy dữ liệu từ bộ dữ liệu của Việt Nam cho hai chỉ số trên trong khoảng thời gian từ năm 2000 đến 2020.
2. Sử dụng biểu đồ đường để thể hiện sự biến động của tỷ lệ tăng trưởng GDP hằng năm qua các năm.
3. Vẽ biểu đồ kết hợp để quan sát xu hướng giữa tăng trưởng GDP và lực lượng lao động, sử dụng trục y kép để dễ dàng so sánh.
4. Tùy chỉnh trục, nhãn và tiêu đề cho biểu đồ.
5. Lưu biểu đồ dưới định dạng SVG và hiển thị kết quả.
6. Viết nhận xét.

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

ax2 = ax1.twinx()

ax1.plot(vn['year'], vn['NY_GDP_MKTP_KD_ZG'], label="Tăng trưởng GDP", color='red')
ax2.bar(vn['year'], vn['SL_TLF_TOTL_IN'], label="Lực lượng lao động", alpha=0.4, color='blue')

ax1.set_ylabel('Tăng trưởng GDP (%)')
ax2.set_ylabel('Lực lượng lao động')
ax1.set_xlabel('Năm')
ax1.set_xticks(vn['year']) 
ax1.tick_params(axis='x', rotation=45)  

ax1.set_title(f"Tăng trưởng GDP và Lực lượng lao động tại Việt Nam ({START_YEAR}-{END_YEAR})")

ax1.grid(False)
ax2.grid(False)

ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

**Nhận xét về mối quan hệ giữa lực lượng lao động và tăng trưởng GDP hằng năm ở Việt Nam trong giai đoạn 2000-2020**

- Nhìn chung, lực lượng lao động tăng đều qua các năm nhưng tăng trưởng GDP thì lại biến động mạnh và không ổn định. Điều này cho thấy rằng tăng trưởng GPD không hoàn toàn tỷ lệ thuận với lực lượng lao động.

- Giai đoạn 2005–2009 và 2018-2020 là hai giai đoạn điển hình cho thấy GDP giảm dù lực lượng lao động vẫn tăng hoặc duy trì, cho thấy các yếu tố khác như khủng hoảng kinh tế hay dịch bệnh ảnh hưởng lớn đến tăng trưởng kinh tế.

**Kết luận**
-  lực lượng lao động không phải yếu tố duy nhất quyết định tăng trưởng GDP.

#### **Câu 3:** Tỷ trọng đóng góp của nông nghiệp, công nghiệp và dịch vụ vào GDP thay đổi ra sao trong khoảng 2000 đến 2020 của Việt Nam, Hoa Kỳ và Nhật Bản?

**Thuộc tính được sử dụng**

1. *NV_AGR_TOTL_ZS* - Giá trị gia tăng ngành nông nghiệp (% GDP)
2. *NV_IND_TOTL_ZS* - Giá trị gia tăng ngành công nghiệp (% GDP)
3. Dịch vụ được tính toán: 100 - Nông nghiệp - Công nghiệp

**Các bước thực hiện**

1. Lấy dữ liệu từ bộ dữ liệu của Việt Nam, Hoa Kỳ và Nhật Bản cho hai chỉ số trên trong khoảng thời gian 2000-2020.
2. Tính toán tỷ trọng ngành dịch vụ bằng công thức bù trừ.
3. Vẽ biểu đồ diện tích chồng (stacked area) để trực quan hóa tỷ trọng đóng góp của nông nghiệp, công nghiệp và dịch vụ vào GDP thay đổi trong giai đoạn 2000 đến 2020
4. Tùy chỉnh trục, nhãn và tiêu đề cho biểu đồ.
5. Lưu biểu đồ dưới định dạng SVG và hiển thị kết quả.
6. Viết nhận xét.

In [ ]:
categories = ['Nông nghiệp', 'Công nghiệp', 'Dịch vụ']
years = np.arange(2008, 2021)

fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharey=True)

for ax, (df, country) in zip(axes, [(vn, 'Việt Nam'), (us, 'Mỹ'), (jp, 'Nhật Bản')]):
    agri = df['NV_AGR_TOTL_ZS'].values
    indus = df['NV_IND_TOTL_ZS'].values
    service = 100 - agri - indus

    ax.stackplot(years, agri, indus, service, labels=categories, alpha=0.8)
    ax.set_title(country)
    ax.set_xlim(2008, 2020)
    ax.set_ylim(0, 100)
    ax.set_xlabel('Năm')
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    ax.set_xticks(np.arange(2008, 2021, 5))

axes[0].set_ylabel('Tỷ trọng ngành trong GDP (%)')
axes[0].legend(loc='upper left')
fig.suptitle('Cơ cấu ngành kinh tế (% GDP) giai đoạn 2000–2020', fontsize=16)

plt.tight_layout()
plt.show()

**Nhận xét về tỷ trọng đóng góp của nông nghiệp, công nghiệp và dịch vụ vào GDP thay đổi ra sao trong khoảng 2000 đến 2020 của Việt Nam, Hoa Kỳ và Nhật Bản?**

**Ở Việt Nam**
- Tỷ trọng nông nghiệp giảm mạnh từ ~25% xuống còn ~13%. Ngược lại, dịch vụ tăng rõ rệt, vượt công nghiệp từ năm 2011 và giữ vị trí cao nhất đến năm 2020.
- Công nghiệp có xu hướng tăng nhẹ, dao động quanh mức 35–38%.

**Ở Hoa Kỳ**
- Dịch vụ luôn chiếm ưu thế vượt trội (~80%) và ổn định trong suốt giai đoạn.
- Công nghiệp giảm nhẹ xuống dưới 20%, trong khi nông nghiệp duy trì mức rất thấp (~1%) và gần như không thay đổi.

**Ở Nhật Bản**
- Tương tự với Hoa Kỳ, nông nghiệp duy trì tỷ trọng rất thấp (dưới 2%) trong suốt giai đoạn.
- Dịch vụ giữ mức cao (~70%) và ổn định, trong khi công nghiệp giảm nhẹ nhưng vẫn chiếm khoảng 25–30%.

**Kết luận**
- Các nền kinh tế phát triển như Hoa Kỳ và Nhật Bản có cơ cấu kinh tế tập trung mạnh vào dịch vụ. Trong khi đó, Việt Nam đang trong quá trình chuyển dịch cơ cấu, với xu hướng giảm phụ thuộc vào nông nghiệp và tăng tỷ trọng dịch vụ.

### **Chủ đề Biến đổi khí hậu**

#### **Câu 4:** Biến động tiêu thụ năng lượng điện và năng lượng sơ cấp đầu người ở Hoa Kỳ từ 2000–2020 như thế nào? Có năm nào vượt trội hoặc bất thường không?

**Thuộc tính được sử dụng**

1. *EG_USE_ELEC_KH_PC* – Tiêu thụ điện bình quân đầu người (kWh/người).
2. *EG_USE_PCAP_KG_OE* – Tiêu thụ năng lượng bình quân đầu người (kg dầu tương đương/người).

**Các bước thực hiện**

1. Trích xuất dữ liệu cho 2 chỉ số trên từ năm 2000–2020 từ bộ dữ liệu.
2. Vẽ biểu đồ hộp (box plot) để trực quan hóa sự phân bố của từng chỉ số trong giai đoạn 2000–2020.
3. Phân tích xem có năm nào là ngoại lệ (outlier) hoặc sự biến động có lớn không giữa các năm.
4. Tùy chỉnh trục, nhãn và tiêu đề cho biểu đồ.
5. Lưu biểu đồ dưới định dạng SVG và hiển thị kết quả.
6. Đưa ra nhận xét về mức độ ổn định trong tiêu thụ năng lượng và xu hướng thay đổi trong hai thập kỷ qua.

In [ ]:
fix, axes = plt.subplots(1, 2, figsize=(10, 6))

sns.boxplot(data=us['EG_USE_ELEC_KH_PC'], ax=axes[0], color='skyblue')
axes[0].set_title('Phân phối tiêu thụ điện')
axes[0].set_ylabel('Tiêu thụ điện (kWh/người)')

sns.boxplot(data=us['EG_USE_PCAP_KG_OE'], ax=axes[1], color='lightcoral')
axes[1].set_title('Phân phối tiêu thụ năng lượng')
axes[1].set_ylabel('Tiêu thụ năng lượng (kg dầu/người)')

plt.tight_layout()
plt.show()

**Nhận xét về phân phối tiêu thụ điện và năng lượng sơ cấp bình quân đầu người tại Hoa Kỳ (2000–2020)**

- **Tiêu thụ điện (kWh/người)** có phân phối khá hẹp, dao động chủ yếu trong khoảng **12900–13700 kWh/người**, với **trung vị khoảng 13300 kWh/người**. Không có giá trị ngoại lai rõ ràng, cho thấy mức tiêu thụ điện tương đối ổn định trong giai đoạn này. Khoảng tứ phân vị (IQR) không quá rộng, chứng tỏ sự dao động giữa các năm là nhỏ.

- **Tiêu thụ năng lượng sơ cấp (kg dầu/người)** có phân phối rộng hơn một chút, với giá trị dao động trong khoảng **6800–8100 kg dầu/người**. Trung vị rơi vào khoảng **7600 kg dầu/người**. Có vẻ như **phân phối hơi lệch về phía thấp** khi phần dưới của hộp (Q1–Median) dài hơn phần trên. Điều này phản ánh mức tiêu thụ năng lượng có sự sụt giảm nhẹ trong một vài năm (có thể do khủng hoảng hoặc chuyển dịch sang năng lượng tái tạo).

**Kết luận**

- Mức **tiêu thụ điện bình quân đầu người ở Hoa Kỳ khá ổn định** trong hai thập kỷ qua, phản ánh sự ổn định trong lối sống, hạ tầng và nhu cầu dân cư.

- Trong khi đó, **tiêu thụ năng lượng sơ cấp có xu hướng dao động mạnh hơn**, điều này có thể liên quan đến sự thay đổi trong chính sách năng lượng, công nghệ tiết kiệm nhiên liệu, và xu hướng sử dụng năng lượng tái tạo.

- **Không có giá trị ngoại lai rõ rệt** trong cả hai biểu đồ, cho thấy dữ liệu tương đối 'sạch' và không có biến động đột ngột bất thường.


#### **Câu 5**: Tỷ lệ dân số sống ở khu vực đô thị của Hoa Kỳ, Nhật Bản và Việt Nam đã thay đổi như thế nào từ năm 2000 đến năm 2020?
**Thuộc tính được sử dụng**

1. *SP_URB_TOTL_IN_ZS* – Tiêu thụ điện bình quân đầu người (kWh/người)

**Các bước thực hiện**

1. Trích xuất dữ liệu cho chỉ số *SP_URB_TOTL_IN_ZS* của ba quốc gia Hoa Kỳ (US), Nhật Bản (JP), và Việt Nam (VN) từ năm 2000 đến 2020.
2. Tạo DataFrame gồm các cột: *Year*, *US*, *JP*, *VN* tương ứng với từng quốc gia.
3. Sử dụng biểu đồ miền (area plot) kết hợp đường line để trực quan hóa xu hướng thay đổi theo thời gian.
4. Tùy chỉnh trục, nhãn và tiêu đề cho biểu đồ.
5. Lưu biểu đồ dưới định dạng SVG và hiển thị kết quả.
6. Nhận xét.

In [ ]:
df = pd.DataFrame(data={
    'Year': np.arange(2008, 2021),
    'US': us['SP_URB_TOTL_IN_ZS'],
    'JP': jp['SP_URB_TOTL_IN_ZS'],
    'VN': vn['SP_URB_TOTL_IN_ZS']
})

plt.figure(figsize=(12, 6))
plt.fill_between(df['Year'], df['US'], color="blue", alpha=0.5)
plt.plot(df['Year'], df['US'], color="blue", label='Hoa Kỳ', marker='o')
plt.fill_between(df['Year'], df['JP'], color="green", alpha=0.5)
plt.plot(df['Year'], df['JP'], color="green", label='Nhật Bản', marker='o')
plt.fill_between(df['Year'], df['VN'], color="red", alpha=0.5)
plt.plot(df['Year'], df['VN'], color="red", label='Việt Nam', marker='o')

plt.xticks(df['Year'])
plt.grid(axis='x', linestyle='--', alpha=0.3)
plt.title('Tỉ lệ dân số thành thị (%) tại Hoa Kỳ, Nhật Bản và Việt Nam (2008-2020)')
plt.xlabel('Năm')
plt.ylabel('Tỉ lệ dân số thành thị (%)')
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()

**Nhận xét về tỉ lệ dân số thành thị tại Hoa Kỳ, Nhật Bản và Việt Nam (2000–2020)**

- **Hoa Kỳ** có tỷ lệ dân số thành thị khá **ổn định**, dao động trong khoảng **79% đến 83%**. Đường biểu diễn cho thấy mức tăng nhẹ qua thời gian, phản ánh sự phát triển đều và ổn định về đô thị hóa.

- **Nhật Bản** là quốc gia có **tỷ lệ đô thị hóa cao nhất** trong ba nước, vượt mốc **90%** từ sau năm 2010 và duy trì ở mức **91–92%** đến năm 2020. Quá trình đô thị hóa của Nhật Bản diễn ra nhanh giai đoạn đầu và dần bão hòa.

- **Việt Nam** có tỷ lệ đô thị hóa thấp nhất nhưng lại tăng nhanh nhất, từ mức **~25% năm 2000** lên gần **38% vào năm 2020**. Biểu đồ cho thấy xu hướng tăng rõ rệt, đặc biệt sau năm 2010, phản ánh sự chuyển dịch mạnh mẽ từ nông thôn sang đô thị.

**Kết luận**

- **Nhật Bản** đã đạt mức đô thị hóa rất cao và gần như ổn định trong thập kỷ qua.
- **Hoa Kỳ** giữ mức độ đô thị hóa cao, với đà tăng chậm và đều.
- **Việt Nam** đang trong giai đoạn tăng tốc mạnh về đô thị hóa, thể hiện xu hướng phát triển kinh tế – xã hội nhanh chóng.

#### **Câu 6:** Mối quan hệ giữa quy mô dân số và tổng lượng điện năng tiêu thụ của Việt Nam từ năm 2000 đến 2020 thay đổi như thế nào? Những năm nào cho thấy sự tăng trưởng đột phá về nhu cầu điện tính trên đầu người?

**Thuộc tính được sử dụng**

1. *EG_USE_ELEC_KH_PC* – Tiêu thụ điện bình quân đầu người (kWh/người)
2. *SP_POP_TOTL* – Tổng dân số

**Các bước thực hiện**

1. Trích xuất dữ liệu của Việt Nam từ năm 2000 đến 2020 cho hai chỉ số: dân số (`SP_POP_TOTL`) và tiêu thụ điện bình quân đầu người (`EG_USE_ELEC_KH_PC`).
2. Tính tổng lượng điện năng tiêu thụ bằng cách nhân hai chỉ số.
3. Tạo biểu đồ Bubble Plot:
   - Trục X: Dân số
   - Trục Y: Tiêu thụ điện bình quân đầu người
   - Kích thước bong bóng: Tổng lượng điện tiêu thụ
   - Nhãn từng năm giúp nhận diện các mốc thời gian cụ thể
4. Tùy chỉnh trục, nhãn và tiêu đề cho biểu đồ.
5. Lưu biểu đồ dưới định dạng SVG và hiển thị kết quả.
6. Nhận xét biểu đồ.


In [ ]:
df = pd.DataFrame(data={
    'Year': np.arange(2008, 2021),
    'SP_POP_TOTL': vn['SP_POP_TOTL'],
    'EG_USE_ELEC_KH_PC': vn['EG_USE_ELEC_KH_PC']
})

df['TOTL_ELEC'] = df['SP_POP_TOTL'] * df['EG_USE_ELEC_KH_PC']

plt.figure(figsize=(12, 6))
plt.scatter(df['SP_POP_TOTL'], df['EG_USE_ELEC_KH_PC'], 
            s=df['TOTL_ELEC'] / 100000000.5, 
            alpha=0.5, c='red', edgecolors="w", linewidth=1)

for i in range(len(df)):
    x = df['SP_POP_TOTL'][i]
    y = df['EG_USE_ELEC_KH_PC'][i]
    year = df['Year'][i]

    if np.isfinite(x) and np.isfinite(y):
        plt.text(x, y, str(year), fontsize=9, ha='center', va='center', color='black')


plt.title('Dân số và tiêu thụ điện tại Việt Nam (2008-2020)')
plt.xlabel('Dân số')
plt.ylabel('Tiêu thụ điện/người')
plt.grid()
plt.tight_layout()
plt.show()

**Nhận xét về dân số và tiêu thụ điện tại Việt Nam (2000–2020)**

- **Giai đoạn 2000–2014**: Dữ liệu cho thấy **cả dân số và tiêu thụ điện/người đều tăng mạnh mẽ**.  
  - Dân số tăng từ khoảng **77 triệu (năm 2000)** lên gần **92 triệu (năm 2014)**.  
  - Tiêu thụ điện/người tăng từ khoảng **300 kWh/người lên hơn 1400 kWh/người**.  
  - Điều này cho thấy mức sống và nhu cầu sử dụng điện tăng rõ rệt, phản ánh quá trình **công nghiệp hóa và hiện đại hóa** đất nước.

- **Tốc độ tăng tiêu thụ điện/người vượt xa tốc độ tăng dân số**, cho thấy nhu cầu điện không chỉ đến từ dân cư mà còn từ hoạt động sản xuất, kinh doanh.

- **Kích thước bong bóng** (biểu thị tổng sản lượng điện tiêu thụ) cũng tăng theo thời gian, khẳng định mức tiêu thụ điện toàn quốc tăng mạnh.

- **Xu hướng tăng đều** và ổn định qua từng năm từ 2000 đến 2014, không có năm nào bị gián đoạn hoặc sụt giảm.

**Lưu ý**:  
Do **thiếu dữ liệu về tiêu thụ điện đầu người từ năm 2015 trở đi**, nên biểu đồ **không hiển thị** từ năm 2015 đến 2020.

**Kết luận**

- Việt Nam đang trong quá trình **tăng trưởng mạnh về nhu cầu sử dụng điện**, phù hợp với đà phát triển kinh tế và đô thị hóa.  
- Sự tăng nhanh về tiêu thụ điện/người là chỉ dấu tích cực về **mức sống, công nghệ và công nghiệp hóa** tại Việt Nam trong giai đoạn 2000–2014.

### **Chủ đề Kinh tế & Phát triển**

#### **Câu 7**: Tình hình tăng trưởng GDP của Hoa Kỳ qua các năm (2008-2020)

**Thuộc tính được sử dụng**

1. *year* - Năm từ 2008 đến 2020.
2. *NY_GDP_MKTP_KD_ZG* - Tăng trưởng GDP hàng năm của Hoa Kỳ (% thay đổi so với năm trước).

**Các bước thực hiện**

1. Lấy dữ liệu tăng trưởng GDP hàng năm (`NY_GDP_MKTP_KD_ZG`) của Hoa Kỳ từ bộ dữ liệu cho giai đoạn 2008-2020.
2. Tạo biểu đồ cột ngang bằng cách sử dụng year làm trục y và `NY_GDP_MKTP_KD_ZG` làm trục x.
3. Tùy chỉnh biểu đồ: đặt tiêu đề, nhãn trục, giới hạn trục x dựa trên giá trị lớn nhất của tăng trưởng GDP, thêm lưới trục x, và sử dụng màu sắc có điều kiện để phân biệt tăng trưởng dương/âm.
4. Điều chỉnh bố cục biểu đồ để đảm bảo các nhãn hiển thị đầy đủ và không bị cắt.
5. Hiển thị biểu đồ và đưa ra nhận xét dựa trên xu hướng quan sát được.

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(13, 15), sharex=True)

axes[0].barh(us['year'], us['NY_GDP_MKTP_KD_ZG'], color=np.where(us['NY_GDP_MKTP_KD_ZG'] < 0, '#f28705', '#025bc7'))
axes[0].set_title(label='Tăng trưởng GDP của Hoa Kỳ (2008-2020)')
axes[0].set_xlabel(xlabel='Tăng trưởng GDP (%)')
axes[0].set_ylabel(ylabel='Năm')

axes[0].set_xlim(-max(us['NY_GDP_MKTP_KD_ZG']) * 1.1, max(us['NY_GDP_MKTP_KD_ZG']) * 1.1)
axes[0].set_yticks(ticks=range(2008, 2021), labels=range(2008, 2021))
axes[0].grid(axis='x', linestyle='--', linewidth=0.5)

axes[1].barh(jp['year'], jp['NY_GDP_MKTP_KD_ZG'], color=np.where(jp['NY_GDP_MKTP_KD_ZG'] < 0, '#f28705', '#025bc7'))
axes[1].set_title(label='Tăng trưởng GDP của Nhật Bản (2008-2020)')
axes[1].set_xlabel(xlabel='Tăng trưởng GDP (%)')
axes[1].set_ylabel(ylabel='Năm')

axes[1].set_xlim(-max(jp['NY_GDP_MKTP_KD_ZG']) * 1.1, max(jp['NY_GDP_MKTP_KD_ZG']) * 1.1)
axes[1].set_yticks(ticks=range(2008, 2021), labels=range(2008, 2021))
axes[1].grid(axis='x', linestyle='--', linewidth=0.5)

axes[2].barh(vn['year'], vn['NY_GDP_MKTP_KD_ZG'], color=np.where(vn['NY_GDP_MKTP_KD_ZG'] < 0, '#f28705', '#025bc7'))
axes[2].set_title(label='Tăng trưởng GDP của Việt Nam (2008-2020)')
axes[2].set_xlabel(xlabel='Tăng trưởng GDP (%)')
axes[2].set_ylabel(ylabel='Năm')

axes[2].set_xlim(-max(vn['NY_GDP_MKTP_KD_ZG']) * 1.1, max(vn['NY_GDP_MKTP_KD_ZG']) * 1.1)
axes[2].set_yticks(ticks=range(2008, 2021), labels=range(2008, 2021))
axes[2].grid(axis='x', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()

**Nhận xét**

- Về tổng quan, nền kinh tế Hoa Kỳ phát triển mạnh mẽ khi hầu hết mức tăng trưởng GDP qua các năm đều dương cùng sự phục hồi mạnh mẽ khi mức GPD tăng vọt chỉ sau 1 năm suy thoái.
- Trong tất cả các năm từ 2000 đến 2020, chỉ có 2 năm mức tăng trưởng GDP của Mỹ đạt mức âm, đó là năm 2009 và năm 2020.
    - Năm 2009: Năm này hứng chịu mọi hậu quả của cuộc khủng hoảng tài chính toàn cầu, xuất phát từ việc **Lehman Brothers phá sản** - công tỳ dịch vụ tài chính toàn cầu.
    - Năm 2020: Đại dịch COVID-19 bùng phát dẫn đến phong tỏa diện rộng, tỷ lệ thất nghiệp tăng mạnh (14.7% - mức cao nhất kể từ Đại suy thoái năm 2009) và sụ sụt giảm tiêu dùng.

#### Câu 8: Tỷ lệ xuất nhập khẩu của Hoa Kỳ ảnh hưởng đến sự tăng trưởng GDP như thế nào (2000 - 2020)?

**Thuộc tính được sử dụng**

1. *NE_EXP_GNFS_ZS*-  Tỷ lệ xuất khẩu hàng hóa và dịch vụ (% GDP).
2. *NE_IMP_GNFS_ZS* - Tỷ lệ nhập khẩu hàng hóa và dịch vụ (% GDP).
3. *NY_GDP_MKTP_KD_ZG* - Tăng trưởng GDP hàng năm (% thay đổi so với năm trước)

**Các bước thực hiện**

1. Lấy dữ liệu từ bộ dữ liệu của Hoa Kỳ cho ba chỉ số *NE_EXP_GNFS_ZS*, *NE_IMP_GNFS_ZS* và *NY_GDP_MKTP_KD_ZG* trong giai đoạn 2000-2020.
2. Vẽ biểu đồ phân tán với *NE_EXP_GNFS_ZS* trên trục x, *NE_IMP_GNFS_ZS* trên trục y và sử dụng *NY_GDP_MKTP_KD_ZG* để xác định kích thước và màu sắc của các điểm.
3. Tùy chỉnh biểu đồ: thêm tiêu đề, nhãn trục, chú thích (legend) cho tăng trưởng GDP, và lưới nền để dễ quan sát.
4. Điều chỉnh bố cục biểu đồ để đảm bảo các thành phần hiển thị rõ ràng và không bị cắt.
5. Hiển thị biểu đồ và đưa ra nhận xét về mối quan hệ giữa tỷ lệ xuất nhập khẩu và tăng trưởng GDP dựa trên xu hướng quan sát được.

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12, 18))

sns.scatterplot(x='NE_EXP_GNFS_ZS', y='NE_IMP_GNFS_ZS', size='NY_GDP_MKTP_KD_ZG',
                sizes=(50, 500), data=us, hue='NY_GDP_MKTP_KD_ZG', palette='coolwarm', ax=axes[0])
axes[0].set_title(label='Tỷ lệ xuất khẩu và nhập khẩu hàng hóa và dịch vụ so với sự tăng trưởng GDP của Hoa Kỳ (2008 - 2020)')
axes[0].set_xlabel(xlabel='Tỷ lệ xuất khẩu hàng hóa và dịch vụ (%)')
axes[0].set_ylabel(ylabel='Tỷ lệ nhập khẩu hàng hóa và dịch vụ (%)')
axes[0].legend(title='Tăng trưởng GDP (%)', loc='upper left')
axes[0].grid(linestyle='--', alpha=0.6)

sns.scatterplot(x='NE_EXP_GNFS_ZS', y='NE_IMP_GNFS_ZS', size='NY_GDP_MKTP_KD_ZG',
                sizes=(50, 500), data=us, hue='NY_GDP_MKTP_KD_ZG', palette='coolwarm', ax=axes[1])
axes[1].set_title(label='Tỷ lệ xuất khẩu và nhập khẩu hàng hóa và dịch vụ so với sự tăng trưởng GDP của Nhật Bản (2008 - 2020)')
axes[1].set_xlabel(xlabel='Tỷ lệ xuất khẩu hàng hóa và dịch vụ (%)')
axes[1].set_ylabel(ylabel='Tỷ lệ nhập khẩu hàng hóa và dịch vụ (%)')
axes[1].legend(title='Tăng trưởng GDP (%)', loc='upper left')
axes[1].grid(linestyle='--', alpha=0.6)

sns.scatterplot(x='NE_EXP_GNFS_ZS', y='NE_IMP_GNFS_ZS', size='NY_GDP_MKTP_KD_ZG',
                sizes=(50, 500), data=us, hue='NY_GDP_MKTP_KD_ZG', palette='coolwarm', ax=axes[2])
axes[2].set_title(label='Tỷ lệ xuất khẩu và nhập khẩu hàng hóa và dịch vụ so với sự tăng trưởng GDP của Việt Nam (2008 - 2020)')
axes[2].set_xlabel(xlabel='Tỷ lệ xuất khẩu hàng hóa và dịch vụ (%)')
axes[2].set_ylabel(ylabel='Tỷ lệ nhập khẩu hàng hóa và dịch vụ (%)')
axes[2].legend(title='Tăng trưởng GDP (%)', loc='upper left')
axes[2].grid(linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
us[(us['NE_IMP_GNFS_ZS'] > 17) & (us['NE_EXP_GNFS_ZS'] < 13)]

**Nhận xét**
- Dựa vào biểu đồ trên, ta nhận thấy tỷ lệ xuất khẩu và tỷ lệ nhập khẩu có tương quan thuận, cùng tương quan thuận **nhẹ** với sự tăng trưởng GDP.
- Bên cạnh đó, Hoa Kỳ còn khá phụ thuộc vào nhập khẩu khi tỷ lệ nhập khẩu cao tác động thuận đến mức độ tăng trưởng GDP nhiều hơn tỷ lệ xuất khẩu.
- Đặc biệt, vào năm 2008, khi xảy ra khủng hoảng kinh tế, dù tỷ lệ xuất nhập khẩu khá cao, song vẫn chịu mức tăng trưởng GDP gần 0.

#### Câu 9: Tổng quan nền kinh tế của Hoa Kỳ, Nhật Bản và Việt Nam trong giai đoạn 2008-2020.

**Thuộc tính được sử dụng**

1. *NY_GDP_MKTP_CD* - Tổng GDP (theo USD hiện hành).
2. *NY_GDP_MKTP_KD_ZG* - Tăng trưởng GDP hàng năm (% thay đổi so với năm trước).
3. *NY_GDP_PCAP_CD* - GDP bình quân đầu người (theo USD hiện hành).
4. *NY_GNP_ATLS_CD* - Tổng thu nhập quốc dân (GNI) theo phương pháp Atlas (theo USD hiện hành).

**Các bước thực hiện**

1. Trích xuất dữ liệu từ World Bank cho 12 quốc gia vào năm 2023, với 4 chỉ số: *NY_GDP_MKTP_CD*, *NY_GDP_MKTP_KD_ZG*, *NY_GDP_PCAP_CD*, và *NY_GNP_ATLS_CD*.
2. Chuẩn hóa dữ liệu bằng *StandardScaler* để đảm bảo các chỉ số có cùng thang đo, tránh ảnh hưởng của sự khác biệt về đơn vị.
3. Thử nghiệm thuật toán *KMeans* với số lượng cụm (k) từ 2 đến 10, tính toán *Inertia* và *Silhouette Score* cho từng giá trị k, sau đó vẽ biểu đồ đường để xác định số lượng cụm tối ưu.
4. Chọn k (dựa trên biểu đồ *Inertia* và *Silhouette Score*) và áp dụng KMeans để gom nhóm các quốc gia thành k cụm.
5. Vẽ biểu đồ violin để trực quan hóa phân phối của từng chỉ số kinh tế theo cụm, tùy chỉnh nhãn, tiêu đề và màu sắc.
6. Phân tích đặc điểm của từng cụm và đưa ra gợi ý phát triển phù hợp.

In [ ]:
overview_economic_features = ['NY_GDP_MKTP_CD', 'NY_GDP_PCAP_CD', 'NY_GDP_PCAP_KD_ZG',
                              'NE_EXP_GNFS_ZS', 'BX_KLT_DINV_CD_WD',
                              'NY_GDP_DEFL_KD_ZG', 'FP_CPI_TOTL_ZG',
                              'NE_GDI_TOTL_ZS', 'NY_GNS_ICTR_ZS']

us1 = us.loc[:, overview_economic_features + ['year']]
us1['country'] = 'USA'
jp1 = jp.loc[:, overview_economic_features + ['year']]
jp1['country'] = 'Japan'
vn1 = vn.loc[:, overview_economic_features + ['year']]
vn1['country'] = 'Vietnam'

economic_overview = pd.concat(objs=[us1, jp1, vn1], ignore_index = True)
economic_overview.head()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(13, 10))

for i, col in enumerate(economic_overview.columns[:-2]):
    row_idx, col_idx = divmod(i, 3)
    sns.histplot(data=economic_overview[col], kde=True, ax=axes[row_idx, col_idx])
    axes[row_idx, col_idx].set_title(label=f'{col}')
    axes[row_idx, col_idx].set_xlabel(xlabel='')

plt.suptitle(t='Phân phối các chỉ số kinh tế của Việt Nam, Nhật Bản và Hoa Kỳ (2008-2020)')
plt.tight_layout()
plt.show()

In [ ]:
economic_overview_scaled = economic_overview.copy()
economic_overview_scaled.head()

In [ ]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
economic_overview_scaled.iloc[:, :-2] = pd.DataFrame(data=min_max_scaler.fit_transform(economic_overview.iloc[:, :-2]))
economic_overview_scaled.set_index(keys='country', inplace=True)
economic_overview_scaled.head()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(20, 18), sharey=True)

for i in range(END_YEAR - START_YEAR + 1):
    row, col = divmod(i, 5)
    sns.heatmap(data=economic_overview_scaled[economic_overview_scaled['year'] == START_YEAR + i][overview_economic_features].T,
                vmin=-1, vmax=1, cmap='crest', annot=False, linewidths=0.1, cbar=False, ax=axes[row, col])
    axes[row, col].set_title(label=f'{START_YEAR + i}')
    axes[row, col].set_xlabel(xlabel='')

axes[2, 3].axis('off')
axes[2, 4].axis('off')

fig.colorbar(axes[0, 0].collections[0], ax=axes, anchor=(1.7, 1))
plt.suptitle(t='Phân phối các chỉ số kinh tế của Hoa Kỳ, Nhật Bản và Việt Nam (2008-2020)', fontsize=18)

plt.subplots_adjust(left=0.05, right=0.9, top=0.95, bottom=0)
plt.show()